# Final Entropy-Based Complexity Measures

In [ ]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.feature import graycomatrix
from skimage.measure import shannon_entropy
import json

DATASETS = ["ISIC-2018", "chest_xray", "PolypsSet"]
BASE_PATH = 'path/to/base'

def delentropy(image):
    # Using a 2x2 difference kernel [[-1,+1],[-1,+1]] results in artifacts!
    # In tests the deldensity seemed to follow a diagonal because of the
    # assymetry introduced by the backward/forward difference
    # the central difference correspond to a convolution kernel of
    # [[-1,0,1],[-1,0,1],[-1,0,1]] and its transposed, produces a symmetric
    # deldensity for random noise.
    if True:
        # see paper eq. (4)
        fx = ( image[:,2:] - image[:,:-2] )[1:-1,:]
        fy = ( image[2:,:] - image[:-2,:] )[:,1:-1]
    else:
        # throw away last row, because it seems to show some artifacts which it shouldn't really
        # Cleaning this up does not seem to work
        kernelDiffY = np.array( [ [-1,-1],[1,1] ] )
        fx = signal.fftconvolve( image, kernelDiffY.T ).astype( image.dtype )[:-1,:-1]
        fy = signal.fftconvolve( image, kernelDiffY   ).astype( image.dtype )[:-1,:-1]
    diffRange = np.max( [ np.abs( fx.min() ), np.abs( fx.max() ), np.abs( fy.min() ), np.abs( fy.max() ) ] )
    if diffRange >= 200   and diffRange <= 255  : diffRange = 255
    if diffRange >= 60000 and diffRange <= 65535: diffRange = 65535

    # see paper eq. (17)
    # The bin edges must be integers, that's why the number of bins and range depends on each other
    nBins = min( 1024, 2*diffRange+1 )
    if image.dtype == float:
        nBins = 1024
    # Centering the bins is necessary because else all value will lie on
    # the bin edges thereby leading to assymetric artifacts
    dbin = 0 if image.dtype == float else 0.5
    r = diffRange + dbin
    delDensity, xedges, yedges = np.histogram2d( fx.flatten(), fy.flatten(), bins = nBins, range = [ [-r,r], [-r,r] ] )
    if nBins == 2*diffRange+1:
        assert( xedges[1] - xedges[0] == 1.0 )
        assert( yedges[1] - yedges[0] == 1.0 )

    # Normalization for entropy calculation. np.sum( H ) should be ( imageWidth-1 )*( imageHeight-1 )
    # The -1 stems from the lost pixels when calculating the gradients with non-periodic boundary conditions
    #assert( np.product( np.array( image.shape ) - 1 ) == np.sum( delDensity ) )
    delDensity = delDensity / np.sum( delDensity ) # see paper eq. (17)
    delDensity = delDensity.T
    # "The entropy is a sum of terms of the form p log(p). When p=0 you instead use the limiting value (as p approaches 0 from above), which is 0."
    # The 0.5 factor is discussed in the paper chapter "4.3 Papoulis generalized sampling halves the delentropy"
    H = - 0.5 * np.sum( delDensity[ delDensity.nonzero() ] * np.log2( delDensity[ delDensity.nonzero() ] ) ) # see paper eq. (16)
    return H

def load_image(file_path):
    with Image.open(file_path) as img:
        return np.array(img.convert('L'))

def analyze_dataset(dataset, dataset_path):
    entropies = {'shannon': [], 'glcm': [], 'delentropy': []}
    num_files = len([f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))])

    img_num = 0

    for filename in os.listdir(dataset_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(dataset_path, filename)
            image = load_image(file_path)

            entropies['shannon'].append(shannon_entropy(exposure.rescale_intensity(image, out_range=(0, 255))))
            glcm = graycomatrix(image, distances=[1, 2, 3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
            entropies['glcm'].append(shannon_entropy(glcm))
            entropies['delentropy'].append(delentropy(image))

            print(f'{dataset}: Processed {filename} | File: {img_num+1}/2500')

            img_num += 1

    return entropies

def calculate_stats(data):
    return {
        'mean': np.mean(data),
        'median': np.median(data),
        'std': np.std(data)
    }

def plot_histogram(data, title, filename):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(data, bins=20, color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel('Entropy Value')
    plt.ylabel('Frequency')
    plt.savefig(filename)
    plt.close()


# Main execution
results = {}

for dataset in DATASETS:
    dataset_path = os.path.join(BASE_PATH, dataset, 'preprocessed')
    entropies = analyze_dataset(dataset, dataset_path)

    results[dataset] = {
        entropy_type: calculate_stats(values)
        for entropy_type, values in entropies.items()
    }

    # Plot histograms
    for entropy_type, values in entropies.items():
        plot_histogram(values, f'{dataset} - {entropy_type} Histogram',
              f'path/to/base/{dataset}/{entropy_type}_histogram.png')

# Print results
for dataset, dataset_results in results.items():
    print(f"\nResults for {dataset}:")
    for entropy_type, stats in dataset_results.items():
        print(f"  {entropy_type}:")
        for stat_name, value in stats.items():
            print(f"    {stat_name}: {value:.4f}")

# Save results to file
with open('path/to/base/entropy_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to path/to/base/entropy_results.json")